
# Chapter 15: Distributing TensorFlow Across Devices and Servers

## 1. Pendahuluan

Pada bab ini, fokusnya adalah bagaimana memanfaatkan kemampuan distribusi TensorFlow untuk menjalankan training pada beberapa GPU, beberapa CPU, atau bahkan beberapa server (cluster). Hal ini sangat penting untuk mempercepat training model yang besar, memanfaatkan sumber daya secara efisien, dan memungkinkan eksperimen skala industri.



## 2. Strategi Distribusi

TensorFlow menyediakan API `tf.distribute` untuk mempermudah penanganan distribusi training.

Beberapa strategi umum:
- `MirroredStrategy`: Training paralel pada beberapa GPU dalam satu mesin.
- `MultiWorkerMirroredStrategy`: Training paralel pada beberapa worker node.
- `TPUStrategy`: Optimasi untuk TPU.
- `CentralStorageStrategy`: Bobot disimpan di CPU, worker di GPU.



## 3. Menggunakan MirroredStrategy

`MirroredStrategy` adalah strategi paling sederhana untuk distribusi multi-GPU dalam satu mesin.


In [ ]:

import tensorflow as tf

strategy = tf.distribute.MirroredStrategy()

print("Number of devices: {}".format(strategy.num_replicas_in_sync))

with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(10)
    ])
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])



## 4. MultiWorkerMirroredStrategy

Jika training dilakukan di beberapa mesin (multi-worker cluster), gunakan `MultiWorkerMirroredStrategy`.

Untuk menjalankan cluster, setiap worker harus memiliki environment variable `TF_CONFIG` yang mendefinisikan cluster topology.

Contoh konfigurasi `TF_CONFIG` untuk 2 worker:


In [ ]:

# Contoh JSON TF_CONFIG
{
  "cluster": {
    "worker": ["worker1.example.com:12345", "worker2.example.com:23456"]
  },
  "task": {"type": "worker", "index": 0}
}



## 5. ParameterServerStrategy

`ParameterServerStrategy` digunakan untuk skenario arsitektur parameter server, yang cocok untuk model yang sangat besar. Bobot model disimpan di server terpusat, sedangkan worker menghitung gradien.

Biasanya digunakan pada cluster dengan banyak node. Konsepnya mirip dengan framework distributed lain seperti Horovod.



## 6. TPUStrategy

Untuk memanfaatkan TPU, TensorFlow menyediakan `TPUStrategy` yang secara otomatis mengatur distribusi training di TPU core.

Contoh dasar:


In [ ]:

resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
    # definisi model
    pass



## 7. Dataset Sharding

Saat distribusi training, dataset perlu di-*shard* otomatis untuk setiap replica agar tidak ada overlap data.

TensorFlow menangani hal ini secara otomatis jika dataset dibungkus di dalam scope `strategy`.


In [ ]:

strategy = tf.distribute.MirroredStrategy()
GLOBAL_BATCH_SIZE = 64

dataset = tf.data.Dataset.range(1000)
dataset = dataset.batch(GLOBAL_BATCH_SIZE)

dist_dataset = strategy.experimental_distribute_dataset(dataset)



## 8. Custom Training Loop dengan Distribusi

Untuk custom loop, gunakan `strategy.run()` agar operasi dieksekusi di setiap replica.


In [ ]:

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = tf.keras.Sequential([...])
    optimizer = tf.keras.optimizers.Adam()
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

@tf.function
def train_step(inputs):
    def step_fn(inputs):
        data, labels = inputs
        with tf.GradientTape() as tape:
            predictions = model(data, training=True)
            loss = loss_fn(labels, predictions)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        return loss

    per_replica_losses = strategy.run(step_fn, args=(inputs,))
    return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)



## 9. Tips dan Best Practices

- Gunakan batch size yang lebih besar jika menggunakan banyak device agar throughput optimal.
- Pastikan data pipeline menggunakan `prefetch` dan `cache` untuk meminimalkan bottleneck I/O.
- Monitor resource (GPU/CPU/Network) untuk mendeteksi bottleneck.
- Tes distribusi di subset data dulu sebelum skala penuh.
